In [1]:
#importing dependencies
import nltk
from nltk import bigrams,trigrams 
from nltk.corpus import reuters
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus
from collections import Counter, defaultdict


In [2]:
WikiDatasetPath = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
Sentences = WikiCorpus(WikiDatasetPath).get_texts()
print(Sentences)

<generator object WikiCorpus.get_texts at 0x7f84320b6a40>


In [3]:
nltk.download('punkt')
nltk.download('reuters')
reutersSentences  = reuters.sents()
print(reutersSentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [4]:
def calculateWordProbability(sentenceModel):
  for nextWord in sentenceModel:
    nextWords = sentenceModel[nextWord]
    total_Word_Count = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord]/=total_Word_Count

In [5]:
def calculateSigleWordProbability(sentenceModel,wordCount):
  for word in sentenceModel:
    sentenceModel[word]/=wordCount

In [6]:
#convert to lowercase
def convertToLower(pa):
  if type(pa)==str:
    return pa.lower()
  return pa

In [7]:
sentenceModel4 = defaultdict(lambda: set())
sentenceModel5 = defaultdict(lambda: set())

def calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,sentences):
  wordCount = 0
  for sentence in sentences:
    #print(sentence)
    for word in sentence:
      wordCount+=1
      sentenceModel1[word]+=1
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = convertToLower(previousWord1)
      previousWord2 = convertToLower(previousWord2)
      # print(previousWord1)
      # print(previousWord2)
      nextWord = convertToLower(nextWord)
      sentenceModel2[nextWord][previousWord1]+=1
      sentenceModel3[nextWord][previousWord2]+=1
      sentenceModel4[previousWord1].add(nextWord)
      sentenceModel5[previousWord2].add(nextWord)

  return wordCount

In [8]:
sentenceModel1 = defaultdict(lambda:0)
sentenceModel2 = defaultdict(lambda: defaultdict(lambda:0))
sentenceModel3 = defaultdict(lambda: defaultdict(lambda:0))

In [9]:
wikiWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,Sentences)
print(wikiWordCount)

452944


In [10]:
reutersWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,reutersSentences)
print(reutersWordCount)

1720917


In [13]:
calculateWordProbability(sentenceModel2)


In [14]:
calculateWordProbability(sentenceModel3)


In [15]:
total_word = wikiWordCount + reutersWordCount
calculateSigleWordProbability(sentenceModel1,total_word)

In [16]:
ProbabilityWords = []
def makeWordSuggestionByTrigram(previousWord2,previousWord1):
  for nextWord in sentenceModel4[previousWord1] & sentenceModel5[previousWord2]:
    naiveBiasTrigramWeight = sentenceModel1[nextWord]*sentenceModel2[nextWord][previousWord1]*sentenceModel3[nextWord][previousWord2]
    ProbabilityWords.append((nextWord,naiveBiasTrigramWeight))

In [17]:
makeWordSuggestionByTrigram('my','name')
ProbabilityWords.sort(key=lambda o:o[1],reverse=True)
print(*ProbabilityWords[:10])

('is', 2.684929364088014e-08) ('to', 1.1753701280369164e-09) (',', 5.59438489695973e-10) ('in', 4.6278217257881707e-10) ('and', 4.5063509855288127e-10) ('for', 3.738790783749001e-10) ('would', 3.5383441884561533e-10) ('will', 2.8735045390861837e-10) ('that', 2.0690529880634479e-10) ('or', 1.711572663102755e-10)


In [18]:
while(True):
    text = input("Enter sentence: ")
    if text == "stop":
        print("Program ending...")
        break
    
    else:
        try:
            ProbabilityWords = []
            text = text.split(" ")
            makeWordSuggestionByTrigram(text[0],text[1])
            ProbabilityWords.sort(key=lambda o:o[1],reverse=True)
            print(*ProbabilityWords[:10])
            
        except:
            continue

Enter sentence: hi 

Enter sentence: i am
('afraid', 1.0350247784931974e-06) ('sure', 8.036310732474202e-07) ('astonished', 4.600110126636432e-07) ('speculating', 3.6800881013091455e-07) ('deeply', 1.8818632336239947e-07) ('convinced', 1.5333700422121436e-07) ('confident', 1.2893042223444673e-07) ('hopeful', 4.600110126636432e-08) ('inclined', 4.600110126636432e-08) ('sceptical', 3.833425105530359e-08)
Enter sentence: stop
Program ending...
